In [ ]:
# Example of joint distributions
# author: Amir Farzmahdi
# last update: November 8th, 2024

In [ ]:
# library imports
import math
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import cv2
import matplotlib.cm as cm
from scipy.stats import pearsonr
from scipy.stats import gaussian_kde
from matplotlib.colors import ListedColormap, to_rgb

In [ ]:
# settings

# example image and orientation preference
ori = 3
img_idx = 0
filt_idx = 0
alpha = 15

# font parameters
matplotlib.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['font.size'] = 8

In [ ]:
# load shared and ind gsm models
with open('example_shared_gsm_image_samples.csv', "rb") as fp: 
    shared_model = pickle.load(fp)

gc1_small = shared_model['gc1_small']
gc2_small = shared_model['gc2_small']
n1_spike_small = shared_model['n1_spike_small']
n2_spike_small = shared_model['n2_spike_small']

In [ ]:
def iso_probability_contours_kde_filled(x, y, ax, levels=5, base_color='blue', **kwargs):
    """
    Create a filled contour probability density plot of *x* and *y* using Gaussian Kernel Density Estimation (KDE).
    The regions between contours are filled with varying shades of a single base color, ensuring that areas outside
    the contour levels are not filled.

    Parameters
    ----------
    x, y : array-like, shape (n,)
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the contours into.

    levels : int or array-like
        Number of contour levels or specific contour levels to draw.

    base_color : str or tuple
        The base color for the contours. Can be any Matplotlib color specification.

    **kwargs
        Forwarded to `ax.contourf()`.

    Returns
    -------
    matplotlib.contour.QuadContourSet
        The contour plot object.
    """
    x = np.asarray(x)
    y = np.asarray(y)
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    # Perform Kernel Density Estimation
    xy = np.vstack([x, y])
    kde = gaussian_kde(xy, bw_method= 0.45)

    # Determine the grid range with a margin
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    margin_x = (x_max - x_min) * 0.05
    margin_y = (y_max - y_min) * 0.05
    x_min -= margin_x
    x_max += margin_x
    y_min -= margin_y
    y_max += margin_y

    # Create grid of points
    x_grid = np.linspace(x_min, x_max, 500)
    y_grid = np.linspace(y_min, y_max, 500)
    X, Y = np.meshgrid(x_grid, y_grid)
    
    positions = np.vstack([X.ravel(), Y.ravel()])

    # Compute the KDE on the grid
    Z = np.reshape(kde(positions).T, X.shape)

    # If levels is an integer, generate levels between Z.min() and Z.max()
    if isinstance(levels, int):
        levels = np.linspace(Z.min(), Z.max(), levels)
    
    # Create a colormap with colors progressing from base color to lighter shades
    n_colors = len(levels) - 1  # Number of intervals between levels
    base_rgb = np.array(to_rgb(base_color))
    
    if n_colors > 0:
        factors = (np.arange(n_colors + 1) / n_colors) * 0.5  # Increase brightness
    else:
        factors = np.array([0])
    
    colors = base_rgb + (np.ones(3) - base_rgb) * factors[:, np.newaxis]
    colors = np.clip(colors, 0, 1)  # Ensure RGB values are within [0, 1]
    
    cmap = ListedColormap(colors)


    contourf = ax.contourf(X, Y, Z/np.nanmax(Z), levels, alpha = 0.75, cmap = cmap, extend = 'max')
    
    # Remove edge lines between contour levels
    if contourf is not None:
        for c in contourf.collections:
            c.set_edgecolor("face")
    else:
        print("Contourf returned None — check your input data.")

    return contourf

In [ ]:
# figure3A-topleft
fig, ax = plt.subplots(1, 1, figsize=(1.6, 1.6))

# plot confidence ellipses for small aperture
iso_probability_contours_kde_filled(gc1_small, gc2_small, ax, levels=[0.05, 0.25, 0.45, 0.65, 0.85], base_color=[0.75, 0.75, 0.75])

# adjust axis settings
ax.set_xlabel(r'$\mathregular{g_{c1}}$ (arb. units)', labelpad=2)
ax.set_ylabel(r'$\mathregular{g_{c2}}$ (arb. units)', labelpad=-2)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_yticks([0, 0.4 , 0.8])
ax.set_yticklabels([0, 0.5 , 1])

ax.set_xticks([0.2, 0.6 , 1])
ax.set_xticklabels([0, 0.5, 1])

ax.set_xlim(0.2, 1)
ax.set_ylim(0.0, 0.8)

ax.tick_params(axis='both', labelsize=8, width=1, length=1, pad=3)

# save the plot
plt.savefig('figure1C_left.pdf', bbox_inches='tight', dpi=600)

In [ ]:
n1_spike_small = np.asarray(n1_spike_small, dtype=float)
n2_spike_small = np.asarray(n2_spike_small, dtype=float)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(1.6, 1.6))

corr_small, _ = pearsonr(n1_spike_small, n2_spike_small)
plt.scatter(n1_spike_small[::2], n2_spike_small[::2], s=20, color=[0.75, 0.75, 0.75], linewidths=0.5, edgecolors='w', clip_on=False)

# adjust axis settings
ax.set_xlabel('neuron 1 response', labelpad=2)
ax.set_ylabel('neuron 2 response', labelpad=2)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_yticks([0, 4.5 , 9])
ax.set_yticklabels([0, 5 , 10])

ax.set_xticks([1, 5.5 , 10])
ax.set_xticklabels([0, 5, 10])

ax.set_xlim(1, 10)
ax.set_ylim(0, 9)

ax.tick_params(axis='both', width=1, length=1, pad=3)

# save the plot
plt.savefig('figure1C_right.pdf', bbox_inches='tight', dpi=600)